In [1]:
import hid
import time
import ft260

In [39]:



class FT260HidDriver():

    """
    Key to symbols
    ==============

    S     (1 bit) : Start bit
    P     (1 bit) : Stop bit
    Rd/Wr (1 bit) : Read/Write bit. Rd equals 1, Wr equals 0.
    A, NA (1 bit) : Accept and reverse accept bit.
    Addr  (7 bits): I2C 7 bit address. Note that this can be expanded as usual to
                    get a 10 bit I2C address.
    Comm  (8 bits): Command byte, a data byte which often selects a register on
                    the device.
    Data  (8 bits): A plain data byte. Sometimes, I write DataLow, DataHigh
                    for 16 bit data.
    Count (8 bits): A data byte containing the length of a block operation.

    [..]: Data sent by I2C device, as opposed to data sent by the host adapter.

    More detail documentation is at https://www.kernel.org/doc/Documentation/i2c/smbus-protocol
    """

    def __init__(self, port, device):
        self.port = port
        #self.smbus = smbus
        self.driver_type = 'ft260_hid'
        dev = device
        self.initialize_ftdi()
    


    def initialize_ftdi(self):
        # TODO pripojeni k HID, nyni to mam jako dev
        
        print(f'Device manufacturer: {dev.get_manufacturer_string()}')
        print(f'Product: {dev.get_product_string()}')
        print(f'Serial Number: {dev.get_serial_number_string()}')

        dev.set_nonblocking(0)

        self.reset_i2c()
        #self.set_i2c_speed(100000) # 100 Khz
        self.get_i2c_status()


    def get_i2c_status(self):
        d = dev.get_feature_report(0xC0, 100)

        status = ['busy_chip', 'error', 'no_ack', 'arbitration_lost', 'idle', 'busy_bus']
        bits = [(d[1] & (1 << i)) >> i for i in range(8)]
        status = dict(zip(status, bits))

        baudrate = (d[2] | d[3]<<8)*1000
        status['baudrate'] = baudrate

        return status
        
    
    def reset_i2c(self):
        dev.send_feature_report([0xA1, 0x20])
        
    def set_i2c_speed(self, speed = 100000):
        speed = int(speed/1000)
        LSB = (speed & 0xff)
        MSB = (speed>>8 & 0xff)
        print(f"Nastavit speed na {speed} Hz: ", hex(LSB), hex(MSB))
        dev.send_feature_report([0xA1, 0x22, LSB, MSB])


    def write_byte(self, address, value):
        """
        SMBus Send Byte:  i2c_smbus_write_byte()
        ========================================

        This operation is the reverse of Receive Byte: it sends a single byte
        to a device.  See Receive Byte for more information.

        S Addr Wr [A] Data [A] P

        Functionality flag: I2C_FUNC_SMBUS_WRITE_BYTE
        """

        return None

    def read_byte(self, address):
        """
        SMBus Send Byte:  i2c_smbus_write_byte()
        ========================================

        This operation is the reverse of Receive Byte: it sends a single byte
        to a device.  See Receive Byte for more information.

        S Addr Wr [A] Data [A] P

        Functionality flag: I2C_FUNC_SMBUS_WRITE_BYTE
        """
        return None

    def write_byte_data(self, address, register, value):
        """
        SMBus Read Byte:  i2c_smbus_read_byte_data()
        ============================================

        This reads a single byte from a device, from a designated register.
        The register is specified through the Comm byte.

        S Addr Wr [A] Comm [A] S Addr Rd [A] [Data] NA P

        Functionality flag: I2C_FUNC_SMBUS_READ_BYTE_DATA
        """

        return dev.write([0xD0, address, 0x06, 2, register, value])


    def read_byte_data(self, address, register):
        """
        SMBus Read Byte:  i2c_smbus_read_byte_data()
        ============================================

        This reads a single byte from a device, from a designated register.
        The register is specified through the Comm byte.

        S Addr Wr [A] Comm [A] S Addr Rd [A] [Data] NA P

        Functionality flag: I2C_FUNC_SMBUS_READ_BYTE_DATA
        """


        payload = [0xD0, address, 0x06, 0b01, register]
        dev.write(payload)
        length = (1).to_bytes(2, byteorder='little')
        dev.write([0xC2, address, 0x06, length[0], length[1]])
        d = dev.read(0xde)

        # TODO: Osetrit chyby v chybnem vycteni registru
        return d[2]


    def write_word_data(self, address, register, value):
        """
        SMBus Write Word:  i2c_smbus_write_word_data()
        ==============================================

        This is the opposite of the Read Word operation. 16 bits
        of data is written to a device, to the designated register that is
        specified through the Comm byte.

        S Addr Wr [A] Comm [A] DataLow [A] DataHigh [A] P

        Functionality flag: I2C_FUNC_SMBUS_WRITE_WORD_DATA

        Note the convenience function i2c_smbus_write_word_swapped is
        available for writes where the two data bytes are the other way
        around (not SMBus compliant, but very popular.)
        """
        return dev.write([0xD0, address, 0x06, 3, register, (value)&0xff, (value>>8)&0xff ])

    def read_word_data(self, address, register):
        """
        SMBus Read Word:  i2c_smbus_read_word_data()
        ============================================

        This operation is very like Read Byte; again, data is read from a
        device, from a designated register that is specified through the Comm
        byte. But this time, the data is a complete word (16 bits).

        S Addr Wr [A] Comm [A] S Addr Rd [A] [DataLow] A [DataHigh] NA P

        Functionality flag: I2C_FUNC_SMBUS_READ_WORD_DATA

        Note the convenience function i2c_smbus_read_word_swapped is
        available for reads where the two data bytes are the other way
        around (not SMBus compliant, but very popular.)
        """

        payload = [0xD0, address, 0x06, 0b01, register]
        dev.write(payload)
        length = (2).to_bytes(2, byteorder='little')
        dev.write([0xC2, address, 0x06, length[0], length[1]])
        d = dev.read(0xde)

        # TODO: Osetrit chyby v chybnem vycteni registru
        return d[2]<<8 | d[3]

    def write_block_data(self, address, register, value):
        """
        SMBus Block Write:  i2c_smbus_write_block_data()
        ================================================

        The opposite of the Block Read command, this writes up to 32 bytes to
        a device, to a designated register that is specified through the
        Comm byte. The amount of data is specified in the Count byte.

        S Addr Wr [A] Comm [A] Count [A] Data [A] Data [A] ... [A] Data [A] P

        Functionality flag: I2C_FUNC_SMBUS_WRITE_BLOCK_DATA
        """
        raise NotImplementedError

    def read_block_data(self, address, register):
        """
        SMBus Block Read:  i2c_smbus_read_block_data()
        ==============================================

        This command reads a block of up to 32 bytes from a device, from a
        designated register that is specified through the Comm byte. The amount
        of data is specified by the device in the Count byte.

        S Addr Wr [A] Comm [A]
                   S Addr Rd [A] [Count] A [Data] A [Data] A ... A [Data] NA P

        Functionality flag: I2C_FUNC_SMBUS_READ_BLOCK_DATA
        """
        raise NotImplementedError

    def block_process_call(self, address, register, value):
        """
        SMBus Block Write - Block Read Process Call
        ===========================================

        SMBus Block Write - Block Read Process Call was introduced in
        Revision 2.0 of the specification.

        This command selects a device register (through the Comm byte), sends
        1 to 31 bytes of data to it, and reads 1 to 31 bytes of data in return.

        S Addr Wr [A] Comm [A] Count [A] Data [A] ...
                                     S Addr Rd [A] [Count] A [Data] ... A P

        Functionality flag: I2C_FUNC_SMBUS_BLOCK_PROC_CALL
        """
        raise NotImplementedError

    ### I2C transactions not compatible with pure SMBus driver
    def write_i2c_block(self, address, value):
        """
        Simple send transaction
        ======================

        This corresponds to i2c_master_send.

          S Addr Wr [A] Data [A] Data [A] ... [A] Data [A] P

        More detail documentation is at: https://www.kernel.org/doc/Documentation/i2c/i2c-protocol
        """
        raise NotImplementedError

    def read_i2c_block(self, address, length):
        """
        Simple receive transaction
        ===========================

        This corresponds to i2c_master_recv

          S Addr Rd [A] [Data] A [Data] A ... A [Data] NA P

        More detail documentation is at: https://www.kernel.org/doc/Documentation/i2c/i2c-protocol
        """
        raise NotImplementedError

    # def write_i2c_block_data(self, address, register, value):
    #     """
    #     I2C block transactions do not limit the number of bytes transferred
    #     but the SMBus layer places a limit of 32 bytes.

    #     I2C Block Write:  i2c_smbus_write_i2c_block_data()
    #     ==================================================

    #     The opposite of the Block Read command, this writes bytes to
    #     a device, to a designated register that is specified through the
    #     Comm byte. Note that command lengths of 0, 2, or more bytes are
    #     supported as they are indistinguishable from data.

    #     S Addr Wr [A] Comm [A] Data [A] Data [A] ... [A] Data [A] P

    #     Functionality flag: I2C_FUNC_SMBUS_WRITE_I2C_BLOCK
    #     """
        
    #     payload = [0xD0, address, 0x06, len(value) + 1, register] + value
    #     dev.write(payload)


    # def read_i2c_block_data(self, address, register, length):
    #     data = []
    #     for i in range(length):
    #         self.write_byte_data(address, register, i)
    #         byte = self.read_byte(address)
    #         data.append(byte)
    #     return data
        
    def read_i2c_block_data(self, address, register, length):
        """
        I2C Block Read: i2c_smbus_read_i2c_block_data()
        =================================================

        Reads a block of bytes from a specific register in a device. It's the direct
        opposite of the Block Write command, primarily used for retrieving a series
        of bytes from a given register.

        S Addr Wr [A] Comm [A] S Addr Rd [A] Data [A] Data [A] ... [A] Data [A] P

        The method respects SMBus limitations of 32 bytes for block transactions.
        """

        print(" .. ")
        print("READ BLOCK DATA", address, register, length)

        if length > 60:
            raise ValueError("Attempt to read more than 60 bytes exceeds the limit.")

        read_cmd = [
            0xD0,              # Report ID pro čtení je 0xD0 podle dokumentace
            address,           # 7-bitová I2C adresa a R/W bit nastaven na čtení (1)
            0x02,
            2,  
            0, register,          # Registru, ze kterého chceme číst
        ]
        print("SEND WRITE REQ", [hex(x) for x in read_cmd])
        dev.write(read_cmd)


        # Příprava I2C čtecího požadavku
        length_b = (2).to_bytes(length, byteorder='little')
        read_request = [
            0xC2, 
            address,
            0x04,
            #length_b[0],
            #length_b[1],
            length, 0
            #0xDE,
            #register           # Registru, ze kterého chceme číst
        ]

        print("SEND READ REQ", [hex(x) for x in read_request])

        # Odeslání požadavku pro čtení
        dev.write(read_request)

        # Přijetí dat z I2C zařízení
        received_data = dev.read(length)


        print("RECIEVED", [hex(x) for x in received_data])
        print("RECIEVED", received_data)

        return received_data

    def write_i2c_block_data(self, address, register, data):
        """
        I2C Block Write: i2c_smbus_write_i2c_block_data()
        =================================================

        Writes a block of bytes to a specific register in a device. This command
        is designed for direct I2C communication, allowing for command lengths of 0,
        2, or more bytes, which are indistinguishable from data.

        S Addr Wr [A] Comm [A] Data [A] Data [A] ... [A] Data [A] P

        Functionality flag: I2C_FUNC_SMBUS_WRITE_I2C_BLOCK
        """

        data_length = len(data)
        report_id = 0xD0 + (data_length - 1) // 4

        if data_length > 60:
            raise ValueError("Data block size must not exceed 60 bytes.")

        # Příprava I2C zápisového požadavku
        write_request = [
            report_id,              # Report ID, který odpovídá velikosti dat
            address,           # 7-bitová I2C adresa a W/R bit nastaven na zápis (0)
            0x06,                   # Příznak pro START_AND_STOP
        ] + [data_length] + data   # Délka dat následována daty

        print("SEND WRITE REQ", write_request)
        # Odeslání požadavku pro zápis dat do I2C zařízení
        dev.write(write_request)

        return True
    
    def block_process_call(self, address, register, value):
        if len(value) > 31:
            raise ValueError("Block process call cannot handle more than 31 bytes")
        count = len(value)
        write_payload = [0xD0, address, 0x06, register, count] + value
        dev.write(write_payload)

        # Read the response block
        response_count = dev.read(0xDE, 1)[0]
        response_data = dev.read(0xDE, response_count)
        return response_data





In [40]:
# write byte

address = 0x77


# payload = [0xD0, address, 0x06, 1, 0b00011110]
# dev.write(payload)
# time.sleep(0.2)


value = 0b10100110
value = 0xa6


for value in range(0xa0, 0xae, 2):
    payload = [0xd0, address, 0x06, 1, value]
    dev.write(payload)

    payload = [0xc2, address, 0x06, 2, 0]
    dev.write(payload)

    d = dev.read(0xde, 100)
    d = d[2:d[1]+2]
    data = d[0] << 8 | d[1]
    print(hex(value), data)


while True:
    # write byte 
    value = 0b01001000

    payload = [0xD0, address, 0x06, 1, value]
    dev.write(payload)


    time.sleep(0.1)
    # write byte 

    value = 0b0

    payload = [0xD0, address, 0x06, 1, 0]
    dev.write(payload)


    time.sleep(0.1)

    payload = [0xc2, address, 0x06, 3]
    dev.write(payload)
    data = dev.read(0xde)

    print("PRESSURE")
    data = data[2:data[1]+2]
    data_var = data[0] << 16 | data[1] << 8 | data[2]
    print(data_var)

AttributeError: 'FT260_I2C' object has no attribute 'write'

In [41]:
class read():
    VID = 0x0403
    PID = 0x6030
    VID = 0x1209    
    PID = 0x7aa0
    I2C_INTERFACE = 0

    def __init__(self):
        self.dev = hid.device()
        self.dev.open(self.VID, self.PID)
        print("Connected to HID device", self.dev)


        self.dev.send_feature_report([0xA1, 0x20])
        self.dev.send_feature_report([0xA1, 0x02, 0x01])

        self.ftdi = FT260HidDriver(0, self.dev)




In [42]:
try:
    dev.close()
except:
    pass


dev = hid.device()
dev.open(0x1209, 0x7aa0)


def initialize_ftdi():
    # TODO pripojeni k HID, nyni to mam jako dev

    print(f'Device manufacturer: {dev.get_manufacturer_string()}')
    print(f'Product: {dev.get_product_string()}')
    print(f'Serial Number: {dev.get_serial_number_string()}')

    dev.set_nonblocking(0)

    speed = int(100)
    LSB = (speed & 0xff)
    MSB = (speed>>8 & 0xff)
    print(f"Nastavit speed na {speed} Hz: ", hex(LSB), hex(MSB))
    dev.send_feature_report([0xA1, 0x22, LSB, MSB])


#self.reset_i2c()
#self.set_i2c_speed(100000) # 100 Khz
#self.get_i2c_status()


def get_i2c_status():
    d = dev.get_feature_report(0xC0, 100)

    status = ['busy_chip', 'error', 'no_ack', 'arbitration_lost', 'idle', 'busy_bus']
    bits = [(d[1] & (1 << i)) >> i for i in range(8)]
    status = dict(zip(status, bits))

    baudrate = (d[2] | d[3]<<8)*1000
    status['baudrate'] = baudrate

    print("Status")
    print(status)
    #return status


initialize_ftdi()
get_i2c_status()



Device manufacturer: FTDI
Product: FT260
Serial Number: Љ
Nastavit speed na 100 Hz:  0x64 0x0
Status
{'busy_chip': 0, 'error': 0, 'no_ack': 0, 'arbitration_lost': 0, 'idle': 0, 'busy_bus': 1, 'baudrate': 100000}


unable to receive message
unable to receive message


In [43]:

# c.write_byte_data(addr_switch, 0x01, 0x03)
# zapis do 
dev.write([0xD0, 0x70, 0x06, 2, 0x01, 0x03])

6

In [37]:
address = 0x58
register = 8


register = (0x08).to_bytes(2, byteorder='little')
payload = [0xD4, address, 0x02, 2, register[0], register[1]]
dev.write(payload)
length = (16).to_bytes(2, byteorder='little')
dev.write([0xC2, address, 0x07, length[0], length[1]])
d = dev.read(0xde, 1000)

print("Length", d[1])
print([hex(x) for x in d])
data = 0
for da in d[2:d[1]+2]:
    data = data << 8 | da
print(hex(data))

Length 16
['0xd3', '0x10', '0x9', '0x10', '0x41', '0x8', '0x74', '0x10', '0x8', '0x51', '0xdc', '0xc', '0xa0', '0x80', '0xa0', '0x80', '0x0', '0x59', '0x0', '0x0']
0x910410874100851dc0ca080a0800059


In [38]:
address = 0x50
register = 8


register = (0x08).to_bytes(2, byteorder='little')
payload = [0xD4, address, 0x02, 2, register[0], register[1]]
dev.write(payload)
length = (16).to_bytes(2, byteorder='little')
dev.write([0xC2, address, 0x07, length[0], length[1]])
d = dev.read(0xde, 1000)

print("Length", d[1])
print([hex(x) for x in d])
data = 0
for da in d[2:d[1]+2]:
    data = data << 8 | da
print(hex(data))

Length 16
['0xd3', '0x10', '0xaa', '0x55', '0xde', '0xad', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0x0', '0x0']
0xaa55deadffffffffffffffffffffffff


In [21]:
# Cteni z SHT

temperature_cmd = [0x24, 0x00] 
address = 0x44

register = (0x08).to_bytes(2, byteorder='little')
payload = [0xD4, address, 0x06, 2, temperature_cmd[0], temperature_cmd[1]]
dev.write(payload)
time.sleep(0.2)
length = (6).to_bytes(2, byteorder='little')
dev.write([0xC2, address, 0x06, length[0], length[1]])
d = dev.read(0xde, 1000)
print(d)

[209, 6, 100, 51, 193, 113, 122, 109, 255, 255, 255, 255]


In [22]:
address = 0x53
register = 10
length = 16


register = (0x08).to_bytes(2, byteorder='little')
payload = [0xD4, address, 0x02, 2, register[0], register[1]]
dev.write(payload)
length = (length).to_bytes(2, byteorder='little')
dev.write([0xC2, address, 0x07, length[0], length[1]])
d = dev.read(0xde, 1000)



print("Length", d[1])
print([hex(x) for x in d])
data = 0
for da in d[2:d[1]+2]:
    data = data << 8 | da
print(hex(data))

Length 16
['0xd3', '0x10', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0x9', '0x21']
0xffffffffffffffffffffffffffffffff


In [23]:
address = 0x50
register = 5

data = [0xA, 0xB, 0xC, 0xD, 0xE, 0xF, 0x10, 0x11, 0x12, 0x13, 0x14, 0x15, 0x16, 0x1, 0x1, 0x2]

register = (0x08).to_bytes(2, byteorder='little')
payload = [0xD4, address, 0x02, 2, register[0], register[1]]
dev.write(payload)
length = (len(data)).to_bytes(2, byteorder='little')
dev.write([0xDF, address, 0x07, length[0], length[1]]+ data)
d = dev.read(0xde, 1000)

In [50]:
# S Addr Wr [A] Comm [A] 
#                          Sr Addr Rd [A] [Data] A [Data] A ... A [Data] NA P

get_i2c_status()
addr = 0x58

#dev.send_feature_report([0xA1, 0x40])

# Read request
r = dev.write([
    0xD2, addr, 0x02, 2, 0x00, 0x00
])
time.sleep(0.3)
print(">", r)
r = dev.write([
    0xC2, addr, 0x03, 16, 0 
])

time.sleep(0.3)
#d = dev.read(100, 1000)
d = dev.read(20, 500)

print(">", r)


time.sleep(0.1)
print(d)
get_i2c_status()

sn = 0
for b in d:
    sn = (sn << 8) | b
print(hex(sn))

Status
{'busy_chip': 0, 'error': 0, 'no_ack': 0, 'arbitration_lost': 0, 'idle': 0, 'busy_bus': 1, 'baudrate': 100000}
> -1
> -1
[]
Status
{'busy_chip': 0, 'error': 0, 'no_ack': 0, 'arbitration_lost': 0, 'idle': 0, 'busy_bus': 1, 'baudrate': 100000}
0x0


In [24]:
get_i2c_status()
addr = 0x44
dev.write([
    0xD0, addr, 0x02, 2, 0x24, 0x00,0x00
])
dev.write([
    0xC2, addr, 0x07, 2, 0x24, 0x00
])
time.sleep(0.1)
# dev.write([
#     0xC2, addr, 0x03, 2, 0
# ])

time.sleep(0.1)
d = dev.read(0xff)
print(d)

sn = 0
for b in d:
    sn = (sn << 8) | b
print(hex(sn))

Status
{'busy_chip': 0, 'error': 0, 'no_ack': 0, 'arbitration_lost': 0, 'idle': 0, 'busy_bus': 1, 'baudrate': 100000}
[222, 60, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 64]
0xde3c0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000340


In [47]:
# Buňka: Zápis do EEPROM a následné čtení (address=0x50, register=0x08)
try:
    address = 0x50
    register = 8
    # data to write (example) -- adjust as needed
    data_to_write = [0xAA, 0x55, 0xDE, 0xAD, 0x11, 0x22, 0x33, 0x44]
    print('Writing', [hex(x) for x in data_to_write], 'to EEPROM')
    reg_bytes = (register).to_bytes(2, byteorder='little')
    # Compose write payload: include 2-byte register address then data
    write_data = [reg_bytes[0], reg_bytes[1]] + data_to_write
    data_length = len(write_data)
    if data_length > 60:
        raise ValueError('Write block too large')
    # report_id selection follows existing helper pattern: 0xD0 + (len-1)//4
    report_id = 0xD0 + (data_length - 1) // 4
    write_request = [report_id, address, 0x06, data_length] + write_data
    print('SEND WRITE REQ', [hex(x) for x in write_request])
    dev.write(write_request)
    # small delay to allow EEPROM internal write cycle (adjust if needed)
    time.sleep(0.2)
    # now read back the same number of bytes: set internal address pointer first
    dev.write([0xD4, address, 0x02, 2, reg_bytes[0], reg_bytes[1]])
    time.sleep(0.05)
    # request read using same pattern as other cells (C2, address, 0x07, len_lo, len_hi)
    read_len = len(data_to_write).to_bytes(2, byteorder='little')
    dev.write([0xC2, address, 0x07, read_len[0], read_len[1]])
    d = dev.read(0xde, 1000)
    print('Raw response:', d)
    if isinstance(d, (list, tuple)) and len(d) >= 2:
        payload_len = d[1]
        payload = d[2:2+payload_len]
        print('Read payload bytes:', [hex(x) for x in payload])
        # Compare first N bytes of payload with written data
        if payload[:len(data_to_write)] == data_to_write:
            print('Verify: read matches written data')
        else:
            print('Verify: mismatch — first bytes read:', [hex(x) for x in payload[:len(data_to_write)]])
    else:
        print('Unexpected response format or length:', type(d), len(d) if hasattr(d, '__len__') else 'n/a')
except Exception as e:
    print('EEPROM write/read failed:', e)

Writing ['0xaa', '0x55', '0xde', '0xad', '0x11', '0x22', '0x33', '0x44'] to EEPROM
SEND WRITE REQ ['0xd2', '0x50', '0x6', '0xa', '0x8', '0x0', '0xaa', '0x55', '0xde', '0xad', '0x11', '0x22', '0x33', '0x44']
Raw response: [209, 8, 170, 85, 222, 173, 17, 34, 51, 68, 17, 34]
Read payload bytes: ['0xaa', '0x55', '0xde', '0xad', '0x11', '0x22', '0x33', '0x44']
Verify: read matches written data
Raw response: [209, 8, 170, 85, 222, 173, 17, 34, 51, 68, 17, 34]
Read payload bytes: ['0xaa', '0x55', '0xde', '0xad', '0x11', '0x22', '0x33', '0x44']
Verify: read matches written data


In [46]:
# Diagnostic: small writes (1,2,4,8 bytes) then immediate read — prints raw HID responses
try:
    address = 0x50
    register = 8
    ftdi = FT260HidDriver(0, dev)
    lengths = [1, 2, 4, 8]
    for ln in lengths:
        data = [((0xB0 + i) & 0xFF) for i in range(ln)]
        reg_bytes = (register).to_bytes(2, 'little')
        payload = [reg_bytes[0], reg_bytes[1]] + data
        print('---')
        print(f'Writing {ln} bytes at register 0x{register:02x}:', [hex(x) for x in data])
        # Use class write (which selects report_id internally)
        ftdi.write_i2c_block_data(address, register, payload)
        # wait a bit for internal write
        time.sleep(0.25)
        # set internal pointer explicitly (low-level) and request read
        dev.write([0xD4, address, 0x02, 2, reg_bytes[0], reg_bytes[1]])
        time.sleep(0.05)
        rl = (ln).to_bytes(2, 'little')
        dev.write([0xC2, address, 0x07, rl[0], rl[1]])
        # read raw response from adapter (give timeout)
        raw = dev.read(0xDE, 1000)
        print('Raw read (bytes):', raw)
        if raw and len(raw) >= 2:
            # attempt to extract payload as earlier
            payload_len = raw[1] if isinstance(raw[1], int) else ord(raw[1])
            payload = list(raw[2:2+payload_len])
            print('Parsed payload:', [hex(x) for x in payload])
        else:
            print('Unexpected raw response format:', raw)
except Exception as e:
    print('Diagnostic test failed:', e)

Device manufacturer: FTDI
Product: FT260
Serial Number: Љ
---
Writing 1 bytes at register 0x08: ['0xb0']
SEND WRITE REQ [208, 80, 6, 3, 8, 0, 176]
Raw read (bytes): [208, 1, 176, 1, 0, 0, 176, 0]
Parsed payload: ['0xb0']
---
Writing 2 bytes at register 0x08: ['0xb0', '0xb1']
SEND WRITE REQ [208, 80, 6, 4, 8, 0, 176, 177]
Raw read (bytes): [208, 2, 176, 177, 0, 0, 176, 177]
Parsed payload: ['0xb0', '0xb1']
---
Writing 4 bytes at register 0x08: ['0xb0', '0xb1', '0xb2', '0xb3']
SEND WRITE REQ [209, 80, 6, 6, 8, 0, 176, 177, 178, 179]
Raw read (bytes): [208, 4, 176, 177, 178, 179, 176, 177]
Parsed payload: ['0xb0', '0xb1', '0xb2', '0xb3']
---
Writing 8 bytes at register 0x08: ['0xb0', '0xb1', '0xb2', '0xb3', '0xb4', '0xb5', '0xb6', '0xb7']
SEND WRITE REQ [210, 80, 6, 10, 8, 0, 176, 177, 178, 179, 180, 181, 182, 183]
Raw read (bytes): [209, 8, 176, 177, 178, 179, 180, 181, 182, 183, 180, 181]
Parsed payload: ['0xb0', '0xb1', '0xb2', '0xb3', '0xb4', '0xb5', '0xb6', '0xb7']


In [47]:
dev.close()

In [48]:
FT260HidDriver = ft260.FT260( VID=0x1209, PID=0x7aa0 )
print(FT260HidDriver)

FT260 device with VID: 4617, PID: 31392, SN: Љ


unable to receive message
unable to receive message


In [49]:
dev = FT260HidDriver.FT260_I2C()

In [54]:
sn = dev.read_i2c_block_data(0x58, 0x08, 18)
val = 0
for b in sn:
    val = (val << 8) | b
print(f"SN: 0x{val:0{len(sn)*2}x}")


[212, 18, 9, 16, 65, 8, 116, 16, 8, 81, 220, 12, 160, 128, 160, 128, 0, 89, 0, 0, 17, 1, 0, 1]
SN: 0x0910410874100851dc0ca080a0800059


In [53]:
print(hex(dev.read_byte_data(0x70, 0x01)))
dev.write_byte_data(0x70, 0x01, 0x03)
print(hex(dev.read_byte_data(0x70, 0x01)))

0xb
0xb


In [61]:
ee = dev.read_i2c_block_data(0x50, 0x00, 120)
print([hex(x) for x in ee])

[222, 60, 170, 187, 204, 221, 238, 255, 17, 34, 51, 68, 85, 102, 119, 136, 153, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 34, 51, 68, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 17, 34, 51, 68, 85, 102, 255, 255, 255, 255, 255, 255, 3, 64]
['0xaa', '0xbb', '0xcc', '0xdd', '0xee', '0xff', '0x11', '0x22', '0x33', '0x44', '0x55', '0x66', '0x77', '0x88', '0x99', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x11', '0x22', '0x33', '0x44', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0xff', '0x11', '0x22', '0x33', '0x44', '0x55', '0x66', '0xff', '0xff', '0xff', '0xff']


In [58]:
dev.write_i2c_block_data(0x50, 0x00, [0xAA, 0xBB, 0xCC, 0xDD, 0xEE, 0xFF, 0x11, 0x22, 0x33, 0x44, 0x55, 0x66, 0x77, 0x88, 0x99, 0x00]   )

True